In [5]:
import numpy as np

# Example dataset
data = np.array([
    [0, 1, 0],
    [1, 1, 1],
    [0, 0, 0],
    [1, 0, 1]
])

class DecisionTree:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.tree = None

    def gini_impurity(self, y):
        """
        Calculate Gini Impurity.
        y: array of labels
        """
        unique, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities ** 2)
        return gini

    def split_data(self, X, y, feature_index, threshold):
        """
        Split the dataset based on a feature and a threshold.
        Returns the left and right splits.
        """
        left_mask = X[:, feature_index] <= threshold
        right_mask = X[:, feature_index] > threshold
        return (X[left_mask], y[left_mask]), (X[right_mask], y[right_mask])

    def find_best_split(self, X, y):
        """
        Find the best feature and threshold to split the dataset.
        """
        best_feature = None
        best_threshold = None
        best_impurity = float("inf")
        n_features = X.shape[1]

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                (left_X, left_y), (right_X, right_y) = self.split_data(X, y, feature_index, threshold)

                # Skip if either split is empty
                if len(left_y) == 0 or len(right_y) == 0:
                    continue

                # Weighted Gini Impurity
                left_impurity = self.gini_impurity(left_y)
                right_impurity = self.gini_impurity(right_y)
                weighted_impurity = (
                    len(left_y) / len(y) * left_impurity
                    + len(right_y) / len(y) * right_impurity
                )

                # Update best split
                if weighted_impurity < best_impurity:
                    best_feature = feature_index
                    best_threshold = threshold
                    best_impurity = weighted_impurity

        return best_feature, best_threshold

    def build_tree(self, X, y, depth=0):
        """
        Recursively build the decision tree.
        """
        # Base cases
        if depth >= self.max_depth or len(np.unique(y)) == 1:
            return np.bincount(y).argmax()

        # Find the best split
        feature, threshold = self.find_best_split(X, y)

        # Error: Skip if no split was found (missing base case for stopping recursion)
        if feature is None:
            return np.bincount(y).argmax()  # Returning the most common class

        # Split data
        (left_X, left_y), (right_X, right_y) = self.split_data(X, y, feature, threshold)

        # Build subtrees
        left_subtree = self.build_tree(left_X, left_y, depth + 1)
        right_subtree = self.build_tree(right_X, right_y, depth + 1)

        return {
            "feature": feature,
            "threshold": threshold,
            "left": left_subtree,
            "right": right_subtree,
        }

    def fit(self, X, y):
        """
        Fit the decision tree to the data.
        """
        self.tree = self.build_tree(X, y)

    def predict(self, X):
        """
        Predict labels for the input data.
        """
        def traverse_tree(x, node):
            if isinstance(node, dict):
                if x[node["feature"]] <= node["threshold"]:
                    return traverse_tree(x, node["left"])
                else:
                    return traverse_tree(x, node["right"])
            else:
                return node

        return np.array([traverse_tree(x, self.tree) for x in X])

# Preparing data
X = data[:, :-1]
y = data[:, -1]

# Training the model
model = DecisionTree(max_depth=2)
model.fit(X, y)

# Making predictions
predictions = model.predict(X)
print("Predictions:", predictions)

Predictions: [0 1 0 1]
